In [1069]:
import sympy as sy
from sympy import symbols, Function, diff, Matrix, MatMul, \
    integrate, Symbol, sin, cos, pi, simplify
import pylatex as pltx
from pylatex import Section, Subsection, Command, NoEscape, Package, NewPage
from dewloosh.solid.navier.latex import expr_to_ltx, expr_to_ltx_breqn
from dewloosh.solid.navier.symtools import inv_sym_3x3

In [1070]:
# coordinates in a suitable vector space
x, y, z = symbols('x y z', real=True)

# kinematic variables
w0 = Function('w_0')(x, y)
thx = Function('\\theta_x')(x, y)
thy = Function('\\theta_y')(x, y)

# sign modifiers for kinematic variables
beta_w, beta_u, beta_v, = \
    symbols('\\beta_w, \\beta_u, \\beta_v', real = True)

Displacement Field

In [1071]:
"""
u = -z * beta_u * thx
v = -z * beta_v * thy
w = beta_w * w0
"""
u = z * thy
v = -z * thx
w = w0
disps = [u, v, w]
Matrix(disps)

Matrix([
[ z*\theta_y(x, y)],
[-z*\theta_x(x, y)],
[        w_0(x, y)]])

Boundary Conditions and Trial Solution

In [1072]:
mn = m, n = symbols('m n', integer=True, positive=True)
coeffs = Amn, Bmn, Cmn = symbols('A_{mn} B_{mn} C_{mn}', real=True)
shape = Lx, Ly = symbols('L_x, L_y', real=True)
Sm, Sn = sin(m * pi * x / Lx), sin(n * pi * y / Ly)
Cm, Cn = cos(m * pi * x / Lx), cos(n * pi * y / Ly)

In [1073]:
w0_trial = Cmn * Sm * Sn
thx_trial = Amn * Cn * Sm
thy_trial = Bmn * Sn * Cm
trial = {w0 : w0_trial, thx : thx_trial, thy: thy_trial}
disps_trial = [s.subs(trial).expand().doit() for s in [thx_trial, thy_trial, w0_trial]]
Matrix(disps_trial)

Matrix([
[A_{mn}*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)],
[B_{mn}*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)],
[C_{mn}*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)]])

In [1074]:
Matrix([s.subs(trial).expand().doit() for s in disps])

Matrix([
[ B_{mn}*z*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)],
[-A_{mn}*z*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)],
[   C_{mn}*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)]])

Strain - Displacement Equations

In [1075]:
exx = diff(u, x)
eyy = diff(v, y)
exy = diff(u, y) + diff(v, x)
exz = diff(w, x) + diff(u, z)
eyz = diff(w, y) + diff(v, z)
strains = [exx, eyy, exy, exz, eyz]
Matrix(strains)

Matrix([
[                                   z*Derivative(\theta_y(x, y), x)],
[                                  -z*Derivative(\theta_x(x, y), y)],
[-z*Derivative(\theta_x(x, y), x) + z*Derivative(\theta_y(x, y), y)],
[                         \theta_y(x, y) + Derivative(w_0(x, y), x)],
[                        -\theta_x(x, y) + Derivative(w_0(x, y), y)]])

In [1076]:
strains_trial = [s.subs(trial).expand().doit() for s in strains]
Matrix(strains_trial)

Matrix([
[                                                    -pi*B_{mn}*m*z*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[                                                     pi*A_{mn}*n*z*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y],
[-pi*A_{mn}*m*z*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x + pi*B_{mn}*n*z*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[              B_{mn}*sin(pi*n*y/L_y)*cos(pi*m*x/L_x) + pi*C_{mn}*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x],
[             -A_{mn}*sin(pi*m*x/L_x)*cos(pi*n*y/L_y) + pi*C_{mn}*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y]])

A possible definition of curvatures would be:

In [1077]:
kxx = diff(exx, z)
kyy = diff(eyy, z)
kxy = diff(exy, z)
gen_curvatures = [kxx, kyy, kxy, exz, eyz]
Matrix(gen_curvatures)

Matrix([
[                                 Derivative(\theta_y(x, y), x)],
[                                -Derivative(\theta_x(x, y), y)],
[-Derivative(\theta_x(x, y), x) + Derivative(\theta_y(x, y), y)],
[                     \theta_y(x, y) + Derivative(w_0(x, y), x)],
[                    -\theta_x(x, y) + Derivative(w_0(x, y), y)]])

In [1078]:
gen_curvatures_trial = [k.subs(trial).expand().doit() for k in gen_curvatures]
Matrix(gen_curvatures_trial)

Matrix([
[                                                  -pi*B_{mn}*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[                                                   pi*A_{mn}*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y],
[-pi*A_{mn}*m*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x + pi*B_{mn}*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[          B_{mn}*sin(pi*n*y/L_y)*cos(pi*m*x/L_x) + pi*C_{mn}*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x],
[         -A_{mn}*sin(pi*m*x/L_x)*cos(pi*n*y/L_y) + pi*C_{mn}*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y]])

strains '+' Hooke model -> stresses

In [1079]:
# sign modifiers for internal forces
"""
inds = ['x', 'y', 'xy', 'xz', 'yz']
syms = " ".join(["\\alpha_{}".format('{' + i + '}') for i in inds])
alpha = symbols(sims, real = True)
mx *= alpha[0]
my *= alpha[1]
mxy *= alpha[2]
vx *= alpha[3]
vy *= alpha[4]
"""

C11, C22, C12, C16, C26, C66 = C126ij = \
    symbols('C_11 C_22 C_12 C_16 C_26 C_66', real=True)
C126 = sy.Matrix([[C11, C12, 0], [C12, C22, 0], [0, 0, C66]])
sxx, syy, sxy = MatMul(C126, sy.Matrix([exx, eyy, exy])).doit()
C44, C55 = C45ij = symbols('C_44 C_55', real=True)
C45 = sy.Matrix([[C44, 0], [0, C55]])
syz, sxz = MatMul(C45, sy.Matrix([eyz, exz])).doit()

# integrate through the thickness
h = Symbol('t', real = True)  # thickness
Int = lambda expr : integrate(expr, (z, -h/2, h/2))
D11, D22, D12, D66 = Dij = symbols('D_11 D_22 D_12 D_66', real = True)
S44, S55, = Sij = symbols('S_44 S_55', real = True)
#
mx, my, mxy = M = Matrix([Int(s * z) for s in [sxx, syy, sxy]])
vx, vy = V = Matrix([Int(s) for s in [sxz, syz]])
#
mx = mx.simplify().expand()
cD11 = mx.coeff(C11 * h**3 / 12)
cD12 = mx.coeff(C12 * h**3 / 12)
mx = D11 * cD11 + D12 * cD12
#
my = my.simplify().expand()
cD22 = my.coeff(C22 * h**3 / 12)
cD21 = my.coeff(C12 * h**3 / 12)
my = D22 * cD22 + D12 * cD21
#
mxy = mxy.simplify().expand()
cD66 = mxy.coeff(C66 * h**3 / 12)
mxy = D66 * cD66
#
vx = vx.simplify().expand()
cS55 = vx.coeff(C55 * h)
vx = S55 * cS55
#
vy = vy.simplify().expand()
cS44 = vy.coeff(C44 * h)
vy = S44 * cS44
mx = mx.simplify().expand()
my = my.simplify().expand()
mxy = mxy.simplify().expand()
vx = vx.simplify().expand()
vy = vy.simplify().expand()

In [1080]:
gen_forces = [mx, my, mxy, vx, vy]
Matrix(gen_forces)

Matrix([
[ D_11*Derivative(\theta_y(x, y), x) - D_12*Derivative(\theta_x(x, y), y)],
[ D_12*Derivative(\theta_y(x, y), x) - D_22*Derivative(\theta_x(x, y), y)],
[-D_66*Derivative(\theta_x(x, y), x) + D_66*Derivative(\theta_y(x, y), y)],
[                     S_55*\theta_y(x, y) + S_55*Derivative(w_0(x, y), x)],
[                    -S_44*\theta_x(x, y) + S_44*Derivative(w_0(x, y), y)]])

In [1081]:
gen_forces_trial = [k.subs(trial).expand().doit() for k in gen_forces]
Matrix(gen_forces_trial)

Matrix([
[ pi*A_{mn}*D_12*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y - pi*B_{mn}*D_11*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[ pi*A_{mn}*D_22*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y - pi*B_{mn}*D_12*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[-pi*A_{mn}*D_66*m*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x + pi*B_{mn}*D_66*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[          B_{mn}*S_55*sin(pi*n*y/L_y)*cos(pi*m*x/L_x) + pi*C_{mn}*S_55*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x],
[         -A_{mn}*S_44*sin(pi*m*x/L_x)*cos(pi*n*y/L_y) + pi*C_{mn}*S_44*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y]])

In [1082]:
D = sy.zeros(5, 5)
D[0, 0] = mx.simplify().coeff(kxx)
D[0, 1] = mx.simplify().coeff(kyy)
D[0, 2] = mx.simplify().coeff(kxy)
D[1, 0] = my.simplify().coeff(kxx)
D[1, 1] = my.simplify().coeff(kyy)
D[1, 2] = my.simplify().coeff(kxy)
D[2, 0] = mxy.simplify().coeff(kxx)
D[2, 1] = mxy.simplify().coeff(kyy)
D[2, 2] = mxy.simplify().coeff(kxy)
D[3, 3] = vx.simplify().coeff(exz)
D[3, 4] = vx.simplify().coeff(eyz)
D[4, 3] = vy.simplify().coeff(exz)
D[4, 4] = vy.simplify().coeff(eyz)
D

Matrix([
[D_11, D_12,    0,    0,    0],
[D_12, D_22,    0,    0,    0],
[   0,    0, D_66,    0,    0],
[   0,    0,    0, S_55,    0],
[   0,    0,    0,    0, S_44]])

 Equilibrium equations. The signs of the equations
 is selected such, that the coefficients of the load functions on the
 right-hand sides admit positive signs according to global axes.

In [1083]:
# moment around global X
lhs_mx = simplify(-diff(mxy, x) - diff(my, y) + vy).expand()
# moment around global Y
lhs_my = simplify(-diff(mxy, y) - diff(mx, x) + vx).expand()
# vertical equilibrium
lhs_fz = simplify(-diff(vx, x) - diff(vy, y)).expand()

In [1084]:
equations = [lhs_mx, lhs_my, lhs_fz]
Matrix([lhs_mx, lhs_my, lhs_fz])

Matrix([
[-D_12*Derivative(\theta_y(x, y), x, y) + D_22*Derivative(\theta_x(x, y), (y, 2)) + D_66*Derivative(\theta_x(x, y), (x, 2)) - D_66*Derivative(\theta_y(x, y), x, y) - S_44*\theta_x(x, y) + S_44*Derivative(w_0(x, y), y)],
[-D_11*Derivative(\theta_y(x, y), (x, 2)) + D_12*Derivative(\theta_x(x, y), x, y) - D_66*Derivative(\theta_y(x, y), (y, 2)) + D_66*Derivative(\theta_x(x, y), x, y) + S_55*\theta_y(x, y) + S_55*Derivative(w_0(x, y), x)],
[                                                                       S_44*Derivative(\theta_x(x, y), y) - S_44*Derivative(w_0(x, y), (y, 2)) - S_55*Derivative(\theta_y(x, y), x) - S_55*Derivative(w_0(x, y), (x, 2))]])

In [1085]:
equations_trial = [k.subs(trial).expand().doit() for k in equations]
Matrix(equations_trial)

Matrix([
[-pi**2*A_{mn}*D_22*n**2*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y**2 - pi**2*A_{mn}*D_66*m**2*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x**2 - A_{mn}*S_44*sin(pi*m*x/L_x)*cos(pi*n*y/L_y) + pi**2*B_{mn}*D_12*m*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y) + pi**2*B_{mn}*D_66*m*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y) + pi*C_{mn}*S_44*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[-pi**2*A_{mn}*D_12*m*n*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x*L_y) - pi**2*A_{mn}*D_66*m*n*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x*L_y) + pi**2*B_{mn}*D_11*m**2*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x**2 + pi**2*B_{mn}*D_66*n**2*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_y**2 + B_{mn}*S_55*sin(pi*n*y/L_y)*cos(pi*m*x/L_x) + pi*C_{mn}*S_55*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x],
[                                                                                                                           -pi*A_{mn}*S_44*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y + pi*B_{mn}*S_55*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x + pi**2*C_{mn}*S_44*n**2*sin(pi*

Substitute trial solution

In [1086]:
eq_mx_trial = equations_trial[0].expand().doit() / (Sm * Cn)
eq_mx_trial = eq_mx_trial.simplify().expand()
eq_my_trial = equations_trial[1].expand().doit() / (Cm * Sn)
eq_my_trial = eq_my_trial.simplify().expand()
eq_fz_trial = equations_trial[2].expand().doit() / (Sm * Sn)
eq_fz_trial = eq_fz_trial.simplify().expand()
Matrix([eq_mx_trial, eq_my_trial, eq_fz_trial])

Matrix([
[-pi**2*A_{mn}*D_22*n**2/L_y**2 - pi**2*A_{mn}*D_66*m**2/L_x**2 - A_{mn}*S_44 + pi**2*B_{mn}*D_12*m*n/(L_x*L_y) + pi**2*B_{mn}*D_66*m*n/(L_x*L_y) + pi*C_{mn}*S_44*n/L_y],
[-pi**2*A_{mn}*D_12*m*n/(L_x*L_y) - pi**2*A_{mn}*D_66*m*n/(L_x*L_y) + pi**2*B_{mn}*D_11*m**2/L_x**2 + pi**2*B_{mn}*D_66*n**2/L_y**2 + B_{mn}*S_55 + pi*C_{mn}*S_55*m/L_x],
[                                                           -pi*A_{mn}*S_44*n/L_y + pi*B_{mn}*S_55*m/L_x + pi**2*C_{mn}*S_44*n**2/L_y**2 + pi**2*C_{mn}*S_55*m**2/L_x**2]])

Coefficient matrix

In [1087]:
P = sy.zeros(3, 3)
P[0, :] = Matrix([eq_mx_trial.coeff(c) for c in coeffs]).T
P[1, :] = Matrix([eq_my_trial.coeff(c) for c in coeffs]).T
P[2, :] = Matrix([eq_fz_trial.coeff(c) for c in coeffs]).T
detP, adjP = inv_sym_3x3(P, as_adj_det = True)
detP = detP.simplify().expand()
adjP.simplify()
P

Matrix([
[-pi**2*D_22*n**2/L_y**2 - pi**2*D_66*m**2/L_x**2 - S_44,    pi**2*D_12*m*n/(L_x*L_y) + pi**2*D_66*m*n/(L_x*L_y),                                   pi*S_44*n/L_y],
[   -pi**2*D_12*m*n/(L_x*L_y) - pi**2*D_66*m*n/(L_x*L_y), pi**2*D_11*m**2/L_x**2 + pi**2*D_66*n**2/L_y**2 + S_55,                                   pi*S_55*m/L_x],
[                                         -pi*S_44*n/L_y,                                          pi*S_55*m/L_x, pi**2*S_44*n**2/L_y**2 + pi**2*S_55*m**2/L_x**2]])

In [1088]:
from sympy import pycode
pycode(P)

'ImmutableDenseMatrix([[-math.pi**2*D_22*n**2/L_y**2 - math.pi**2*D_66*m**2/L_x**2 - S_44, math.pi**2*D_12*m*n/(L_x*L_y) + math.pi**2*D_66*m*n/(L_x*L_y), math.pi*S_44*n/L_y], [-math.pi**2*D_12*m*n/(L_x*L_y) - math.pi**2*D_66*m*n/(L_x*L_y), math.pi**2*D_11*m**2/L_x**2 + math.pi**2*D_66*n**2/L_y**2 + S_55, math.pi*S_55*m/L_x], [-math.pi*S_44*n/L_y, math.pi*S_55*m/L_x, math.pi**2*S_44*n**2/L_y**2 + math.pi**2*S_55*m**2/L_x**2]])'

In [1089]:
from sympy import lambdify

lhs = lambdify([m, n, D11, D12, D22, D66, S44, S55, Lx, Ly], P, 'numpy')

In [1090]:
lhs(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

array([[-20.7392088 ,  19.7392088 ,   3.14159265],
       [-19.7392088 ,  20.7392088 ,   3.14159265],
       [ -3.14159265,   3.14159265,  19.7392088 ]])

Postproc

In [1091]:
mx = mx.subs(trial).expand().doit()
mx = mx.simplify().expand()
my = my.subs(trial).expand().doit()
my = my.simplify().expand()
mxy = mxy.subs(trial).expand().doit()
mxy = mxy.simplify().expand()
vx = vx.subs(trial).expand().doit()
vx = vx.simplify().expand()
vy = vy.subs(trial).expand().doit()
vy = vy.simplify().expand()
kxx = kxx.subs(trial).expand().doit()
kxx = kxx.simplify().expand()
kyy = kyy.subs(trial).expand().doit()
kyy = kyy.simplify().expand()
kxy = kxy.subs(trial).expand().doit()
kxy = kxy.simplify().expand()
exz = exz.subs(trial).expand().doit()
exz = exz.simplify().expand()
eyz = eyz.subs(trial).expand().doit()
eyz = eyz.simplify().expand()

In [1092]:
pproc = Matrix([w0_trial, thx_trial, thy_trial, kxx, 
                kyy, kxy, exz, eyz, mx, my, mxy, vx, vy]).T
pycode(pproc)

'ImmutableDenseMatrix([[C_{mn}*math.sin(math.pi*m*x/L_x)*math.sin(math.pi*n*y/L_y), A_{mn}*math.sin(math.pi*m*x/L_x)*math.cos(math.pi*n*y/L_y), B_{mn}*math.sin(math.pi*n*y/L_y)*math.cos(math.pi*m*x/L_x), -math.pi*B_{mn}*m*math.sin(math.pi*m*x/L_x)*math.sin(math.pi*n*y/L_y)/L_x, math.pi*A_{mn}*n*math.sin(math.pi*m*x/L_x)*math.sin(math.pi*n*y/L_y)/L_y, -math.pi*A_{mn}*m*math.cos(math.pi*m*x/L_x)*math.cos(math.pi*n*y/L_y)/L_x + math.pi*B_{mn}*n*math.cos(math.pi*m*x/L_x)*math.cos(math.pi*n*y/L_y)/L_y, B_{mn}*math.sin(math.pi*n*y/L_y)*math.cos(math.pi*m*x/L_x) + math.pi*C_{mn}*m*math.sin(math.pi*n*y/L_y)*math.cos(math.pi*m*x/L_x)/L_x, -A_{mn}*math.sin(math.pi*m*x/L_x)*math.cos(math.pi*n*y/L_y) + math.pi*C_{mn}*n*math.sin(math.pi*m*x/L_x)*math.cos(math.pi*n*y/L_y)/L_y, math.pi*A_{mn}*D_12*n*math.sin(math.pi*m*x/L_x)*math.sin(math.pi*n*y/L_y)/L_y - math.pi*B_{mn}*D_11*m*math.sin(math.pi*m*x/L_x)*math.sin(math.pi*n*y/L_y)/L_x, math.pi*A_{mn}*D_22*n*math.sin(math.pi*m*x/L_x)*math.sin(math.pi*n*

In [1093]:
gen_forces = [mx, my, mxy, vx, vy]
Matrix(gen_forces)

Matrix([
[ pi*A_{mn}*D_12*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y - pi*B_{mn}*D_11*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[ pi*A_{mn}*D_22*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y - pi*B_{mn}*D_12*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[-pi*A_{mn}*D_66*m*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x + pi*B_{mn}*D_66*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[          B_{mn}*S_55*sin(pi*n*y/L_y)*cos(pi*m*x/L_x) + pi*C_{mn}*S_55*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x],
[         -A_{mn}*S_44*sin(pi*m*x/L_x)*cos(pi*n*y/L_y) + pi*C_{mn}*S_44*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y]])

In [1094]:
gen_curvatures = [kxx, kyy, kxy, exz, eyz]
Matrix(gen_curvatures)

Matrix([
[                                                  -pi*B_{mn}*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[                                                   pi*A_{mn}*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y],
[-pi*A_{mn}*m*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x + pi*B_{mn}*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[          B_{mn}*sin(pi*n*y/L_y)*cos(pi*m*x/L_x) + pi*C_{mn}*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x],
[         -A_{mn}*sin(pi*m*x/L_x)*cos(pi*n*y/L_y) + pi*C_{mn}*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y]])

In [1095]:
Matrix(disps_trial)

Matrix([
[A_{mn}*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)],
[B_{mn}*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)],
[C_{mn}*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)]])

In [1096]:
# constans vertical load over a rectangular area
q, w, h = symbols('q w h', real = True)
xc, yc = symbols('x_c y_c', real = True)
qmn = (4/(Lx*Ly)) * integrate(q * Sm * Sn, (x, xc - w/2, xc + w/2),
                            (y, yc - h/2, yc + h/2))
qmn_fz = qmn.simplify().expand()

# constans moment of intensity mx, around global axis X,
# over a rectangular area with width w, height h and center (xc, yc)
m_x, w, h = symbols('m_x w h', real = True)
xc, yc = symbols('x_c y_c', real = True)
qmn = (4/(Lx*Ly)) * integrate(m_x * Cm * Sn, (x, xc - w/2, xc + w/2),
                            (y, yc - h/2, yc + h/2))
qmn_mx = qmn.simplify().expand()

# constant moment of intensity my, around global axis Y,
# over a rectangular area with width w, height h and center (xc, yc)
m_y, w, h = symbols('m_y w h', real = True)
xc, yc = symbols('x_c y_c', real = True)
qmn = (4/(Lx*Ly)) * integrate(m_y * Sm * Cn, (x, xc - w/2, xc + w/2),
                            (y, yc - h/2, yc + h/2))
qmn_my = qmn.simplify().expand()

In [1097]:
f = Matrix([qmn_mx, qmn_my, qmn_fz])
f

Matrix([
[16*m_x*sin(pi*m*w/(2*L_x))*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)*cos(pi*m*x_c/L_x)/(pi**2*m*n)],
[16*m_y*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*cos(pi*n*y_c/L_y)/(pi**2*m*n)],
[  16*q*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)/(pi**2*m*n)]])

In [1098]:
lhs = lambdify([m, n, D11, D12, D22, D66, S44, S55, Lx, Ly], P, 'numpy')
lhs(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

array([[-20.7392088 ,  19.7392088 ,   3.14159265],
       [-19.7392088 ,  20.7392088 ,   3.14159265],
       [ -3.14159265,   3.14159265,  19.7392088 ]])

In [1099]:
rhs = lambdify([m, n, xc, yc, w, h, m_x, m_y, q, Lx, Ly], f, 'numpy')
rhs(1, 1, 5, 5, 1, 1, 0, 0, 10, 10, 10)

array([[0.        ],
       [0.        ],
       [0.39672094]])

In [1100]:
pycode(f)

'ImmutableDenseMatrix([[16*m_x*math.sin((1/2)*math.pi*m*w/L_x)*math.sin((1/2)*math.pi*h*n/L_y)*math.sin(math.pi*n*y_c/L_y)*math.cos(math.pi*m*x_c/L_x)/(math.pi**2*m*n)], [16*m_y*math.sin((1/2)*math.pi*m*w/L_x)*math.sin(math.pi*m*x_c/L_x)*math.sin((1/2)*math.pi*h*n/L_y)*math.cos(math.pi*n*y_c/L_y)/(math.pi**2*m*n)], [16*q*math.sin((1/2)*math.pi*m*w/L_x)*math.sin(math.pi*m*x_c/L_x)*math.sin((1/2)*math.pi*h*n/L_y)*math.sin(math.pi*n*y_c/L_y)/(math.pi**2*m*n)]])'

In [1101]:
dofsol = lambdify([m, n, x, y, Lx, Ly, Amn, Bmn, Cmn], Matrix(disps_trial), 'numpy')
dofsol(1, 1, 5, 5, 10, 10, 1, 1, 1)

array([[6.123234e-17],
       [6.123234e-17],
       [1.000000e+00]])

In [1102]:
Matrix(gen_forces)

Matrix([
[ pi*A_{mn}*D_12*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y - pi*B_{mn}*D_11*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[ pi*A_{mn}*D_22*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y - pi*B_{mn}*D_12*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[-pi*A_{mn}*D_66*m*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x + pi*B_{mn}*D_66*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[          B_{mn}*S_55*sin(pi*n*y/L_y)*cos(pi*m*x/L_x) + pi*C_{mn}*S_55*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x],
[         -A_{mn}*S_44*sin(pi*m*x/L_x)*cos(pi*n*y/L_y) + pi*C_{mn}*S_44*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y]])

In [1103]:
intfors = lambdify([m, n, x, y, D11, D12, D22, D66, S44, S55, Lx, Ly, Amn, Bmn, Cmn], 
                   Matrix(gen_forces), 'numpy')
intfors(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

array([[ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [-5.07198819e-16],
       [-2.62269459e-16]])

In [1104]:
import numpy as np

size = Lx, Ly = (6, 8)
E = 2.89e7
nu = 0.2
t = 0.25

G = E/2/(1+nu)
D = np.array([[1, nu, 0], [nu, 1, 0], [0., 0, (1-nu)/2]]) * t**3 * (E / (1-nu**2)) / 12
S = np.array([[G, 0], [0, G]]) * t
D11, D12, D22, D66 = D[0, 0], D[0, 1], D[1, 1], D[2, 2]
S44, S55 = S[0, 0], S[1, 1]
xc, yc = size[0]/2, size[1]/2
w, h = size[0]/2, size[1]/2
m_x, m_y, p_z = 0, 0, -1000

In [1105]:
M, N = 50, 50
res = np.zeros((M*N, 3))
for i in range(M):
    for j in range(N):
        b_mn = rhs(i+1, j+1, xc, yc, w, h, m_x, m_y, p_z, Lx, Ly)
        A_mn = lhs(i+1, j+1, D11, D12, D22, D66, S44, S55, Lx, Ly)
        res[i*N + j] = np.linalg.solve(A_mn, b_mn).flatten()

In [1106]:
xp, yp = Lx/2, Ly/2
dp = np.zeros(3)
for i in range(M):
    for j in range(N):
        Amn, Bmn, Cmn = res[i*N + j]
        dp += dofsol(i+1, j+1, xp, yp, Lx, Ly, Amn, Bmn, Cmn).flatten()
dp

array([-1.48502015e-18,  2.43889642e-18, -1.16617626e-01])

In [1107]:
from dewloosh.geom import grid, PolyData
from dewloosh.geom.tri.trimesh import triangulate
from dewloosh.geom.topo.tr import Q4_to_T3

shape = nx, ny = (10, 10)
gridparams = {
    'size' : size,
    'shape' : shape,
    'origo' : (0, 0),
    'start' : 0,
    'eshape' : 'Q4'
    }
coords_, topo = grid(**gridparams)
coords = np.zeros((coords_.shape[0], 3))
coords[:, :2] = coords_[:, :]
del coords_
coords, triangles = Q4_to_T3(coords, topo)

triobj = triangulate(points=coords[:, :2], triangles=triangles)[-1]
Mesh = PolyData(coords=coords, topo=triangles)
centers = Mesh.centers()

In [1108]:
nP = len(centers)
data = np.zeros((nP, 3))
for i in range(M):
    for j in range(N):
        Amn, Bmn, Cmn = res[i*N + j]
        for k in range(nP):
            xp, yp = centers[k, :2]
            data[k] += dofsol(i+1, j+1, xp, yp, Lx, Ly, Amn, Bmn, Cmn).flatten()

In [ ]:
%matplotlib qt

In [ ]:
from dewloosh.geom.tri.triplot import triplot
import matplotlib.pyplot as plt
from matplotlib import gridspec

plt.style.use('default')
fig = plt.figure(figsize=(8, 3))  # in inches
fig.patch.set_facecolor('white')
cmap = 'jet'
gs = gridspec.GridSpec(1, 4)

# mesh
ax = fig.add_subplot(gs[0])
triplot(triobj, ax=ax, fig=fig)
# rotX
ax = fig.add_subplot(gs[1])
triplot(triobj, ax=ax, fig=fig, title=r'$\theta_x$',
        data=data[:, 0], cmap=cmap, axis='off')
# rotY
ax = fig.add_subplot(gs[2])
triplot(triobj, ax=ax, fig=fig, title=r'$\theta_y$',
        data=data[:, 1], cmap=cmap, axis='off')
# w
ax = fig.add_subplot(gs[3])
triplot(triobj, ax=ax, fig=fig, title=r'$w$',
        data=data[:, 2], cmap=cmap, axis='off')

fig.tight_layout()

In [ ]:
nP = len(centers)
sdata = np.zeros((nP, 5))
for i in range(M):
    for j in range(N):
        Amn, Bmn, Cmn = res[i*N + j]
        for k in range(nP):
            xp, yp = centers[k, :2]
            sdata[k] += intfors(i+1, j+1, xp, yp, D11, D12, D22, D66, S44, \
                S55, Lx, Ly, Amn, Bmn, Cmn).flatten()

In [ ]:
fig = plt.figure(figsize=(10, 2))  # in inches
fig.patch.set_facecolor('white')
cmap = 'jet'
gs = gridspec.GridSpec(1, 5)

# rotX
ax = fig.add_subplot(gs[0])
triplot(triobj, ax=ax, fig=fig, title=r'$m_x$',
        data=sdata[:, 0], cmap=cmap, axis='off')
# rotY
ax = fig.add_subplot(gs[1])
triplot(triobj, ax=ax, fig=fig, title=r'$m_y$',
        data=sdata[:, 1], cmap=cmap, axis='off')
# w
ax = fig.add_subplot(gs[2])
triplot(triobj, ax=ax, fig=fig, title=r'$m_{xy}$',
        data=sdata[:, 2], cmap=cmap, axis='off')

# rotY
ax = fig.add_subplot(gs[3])
triplot(triobj, ax=ax, fig=fig, title=r'$v_x$',
        data=sdata[:, 3], cmap=cmap, axis='off')
# w
ax = fig.add_subplot(gs[4])
triplot(triobj, ax=ax, fig=fig, title=r'$v_y$',
        data=sdata[:, 4], cmap=cmap, axis='off')

fig.tight_layout()

# **Report Generation with $\LaTeX$**

In [ ]:
geometry_options = {
        "tmargin" : "1.5cm",
        "lmargin" : "1.5cm",
        "rmargin" : "1.5cm"
    }

doc = pltx.Document(geometry_options=geometry_options)

r"""
Tools related to displaying math. It's a bit like the numpy of latex, must
have stuff.
"""
doc.packages.append(Package('amsmath'))

r"""
Sympy uses the 'operatorname' command frequently to print symbols.
"""
doc.packages.append(Package('amsopn'))

r"""
To automatically break long equations into multiple lines.
"""
doc.packages.append(Package('breqn'))

r"""
mathtools provides us with the \coloneqq command, for defining equality
symbol ':='
"""
doc.packages.append(Package('mathtools'))

r"""
Misc
"""
doc.packages.append(Package('enumitem'))  # to customize enumerations
doc.packages.append(Package('xcolor'))  # colors
doc.packages.append(Package('lmodern'))  # high quality fonts

title = "Calculation of rectangular, simply supported plates \
    resting on elastic foundation."
doc.preamble.append(Command('title', title))
doc.preamble.append(Command('author', 'Claude Louis Marie Henri Navier'))
doc.preamble.append(Command('date', NoEscape(r'\today')))
doc.append(NoEscape(r'\maketitle'))
doc.append(NoEscape(r'\tableofcontents'))
doc.append(NewPage())

sections = {}

In [ ]:
sections['theory'] = doc.create(Section('Theoretical background'))

In [ ]:
with sections['theory']:
    
    # displacement field
    with doc.create(Subsection('Displacement Field and Boundary Conditions')):
        doc.append(NoEscape(
            r"""
            \noindent
            The Mindlin-Reissner kinematical model poses the following
            assumptions on the displacement field:
            """))
        doc.append(NoEscape(
            r"""
            \begin{enumerate}[label*=\protect\fbox{MR\arabic{enumi}},
            itemindent=5em]
            \item   normals to the midsurface remain normal
            \item   the plate thickness does not change during deformation
            \item   normal stress through the thickness is ignored
            \end{enumerate}
            """))
        doc.append(expr_to_ltx(r'u(x, y, z)', sy.latex(u)))
        doc.append(expr_to_ltx(r'v(x, y, z)', sy.latex(v)))
        doc.append(expr_to_ltx(r'w(x, y, z)', sy.latex(w)))
        
    # Boundary Conditions and Trial Solution
    with doc.create(Subsection('Boundary Conditions and Trial Solution')):
        doc.append(NoEscape(
            r"""
            All of the boundary conditions are of the essential type, and
            are satisfied a priori by the proper selection of approximation
            functions. Thus, the unknown field functions are sought in the
            form
            """))
        sumltx = r'\sum_{m, n} '
        doc.append(expr_to_ltx(sy.latex(w0), sumltx + sy.latex(w0_trial)))
        doc.append(expr_to_ltx(sy.latex(thx), sumltx + sy.latex(thx_trial)))
        doc.append(expr_to_ltx(sy.latex(thy), sumltx + sy.latex(thy_trial)))
        
    # engineering strains
    with doc.create(Subsection('Strain - Displacement Equations')):
        doc.append(expr_to_ltx(r'\varepsilon_x(x, y, z)', sy.latex(exx)))
        doc.append(expr_to_ltx(r'\varepsilon_y(x, y, z)', sy.latex(eyy)))
        doc.append(expr_to_ltx(r'\gamma_{xy}(x, y, z)', sy.latex(exy)))
        doc.append(expr_to_ltx(r'\gamma_{xz}(x, y, z)', sy.latex(exz)))
        doc.append(expr_to_ltx(r'\gamma_{yz}(x, y, z)', sy.latex(eyz)))
        
        #doc.append('Curvatures are defined as')
        #doc.append(
        #    expr_to_ltx(r"""\kappa_x(x, y) \coloneqq \frac{\partial
        #    \varepsilon_x}{\partial z}""", sy.latex(kxx)))
        #doc.append(
        #    expr_to_ltx(r"""\kappa_y(x, y) \coloneqq \frac{\partial
        #    \varepsilon_y}{\partial z}""", sy.latex(kyy)))
        #doc.append(
        #    expr_to_ltx(r"""\kappa_{xy}(x, y) \coloneqq \frac{\partial
        #    \gamma_{xy}}{\partial z}""", sy.latex(kxy)))
        
    # material model and stress resultants
    with doc.create(Subsection('Stress Resultants')):
        doc.append(expr_to_ltx(r'm_x(x, y)', sy.latex(mx)))
        doc.append(expr_to_ltx(r'm_y(x, y)', sy.latex(my)))
        doc.append(expr_to_ltx(r'm_{xy}(x, y)', sy.latex(mxy)))
        doc.append(expr_to_ltx(r'v_x(x, y)', sy.latex(vx)))
        doc.append(expr_to_ltx(r'v_y(x, y)', sy.latex(vy)))

    # Equilibrium Equations
    with doc.create(Subsection('Equilibrium Equations')):
        doc.append(NoEscape(r'Equilibrium of moments around global $x$ :'))
        doc.append(expr_to_ltx_breqn(sy.latex(lhs_mx), 'm_x(x, y)'))
        doc.append(NoEscape(r'Equilibrium of moments around global $y$ :'))
        doc.append(expr_to_ltx_breqn(sy.latex(lhs_my), 'm_y(x, y)'))
        doc.append(NoEscape(r'Equilibrium of forces along global $z$ :'))
        doc.append(expr_to_ltx_breqn(sy.latex(lhs_fz), 'f_z(x, y)'))

        # substitute trial solution
        doc.append(NoEscape(
            r"""
            \noindent
            Upon substitution of the trial solution into the vertical
            equilibrium equation we obtain the following equality
            """))
        mxltx = r"\left[" + sy.latex(eq_fz_trial.simplify()) + r"\right]" + \
            sy.latex(Sm * Sn)
        doc.append(expr_to_ltx_breqn(mxltx, 'f_z(x, y)'))
        doc.append(NoEscape(
            r"""
            \noindent
            It is clear, that if was possible to write the function
            of vertical forces $f_z(x, y)$ as a finite sum of purely sinusoidal
            functions, the trigonometric terms could be completely ruled out
            from the equation. After forming similar thoughts about the other
            two equations, the loads are sought as
            """))

        # coefficient matrix
        doc.append('Discrete equilibrium equation')
        doc.append(expr_to_ltx(r'\boldsymbol{P}', sy.latex(P), dfrac = True))

    with doc.create(Subsection('Postprocessing')):
        doc.append(expr_to_ltx(r'm_x(x, y)', sy.latex(mx)))
        doc.append(expr_to_ltx(r'm_y(x, y)', sy.latex(my)))
        doc.append(expr_to_ltx(r'm_{xy}(x, y)', sy.latex(mxy)))
        doc.append(expr_to_ltx(r'v_x(x, y)', sy.latex(vx)))
        doc.append(expr_to_ltx(r'v_y(x, y)', sy.latex(vy)))
        doc.append(expr_to_ltx(r'\kappa_x(x, y)', sy.latex(kxx)))
        doc.append(expr_to_ltx(r'\kappa_y(x, y)', sy.latex(kyy)))
        doc.append(expr_to_ltx(r'\kappa_{xy}(x, y)', sy.latex(kxy)))
        doc.append(expr_to_ltx(r'\gamma_{xz}(x, y)', sy.latex(exz)))
        doc.append(expr_to_ltx(r'\gamma_{yz}(x, y)', sy.latex(eyz)))

    with doc.create(Subsection('Loads')):
        doc.append(NoEscape(
            r"""
            If the load functions are represented as suggested by the
            left hand-sides of the equilibrium equations, the solution of
            the system simplifies to the solution of a 3x3 algebraic equation
            system.
            """))

        # constans vertical load over a rectangular area
        doc.append(NoEscape(
            r"""
            \noindent
            \paragraph{Constans vertical load of intensity $q$ over a
            rectangular area}
            """))
        doc.append(expr_to_ltx(r'q_{mn}', sy.latex(qmn_fz), post = ','))
        #doc.append(NoEscape(r"\noindent"))
        doc.append(NoEscape(
            r"""
            where $w$ and $h$ denote the width and height, $x_c$ and $y_c$ the
            coordinates of the center point of the rectangle.
            """))

        # constans moment of intensity mx, around global axis X,
        # over a rectangular area with width w, height h and center (xc, yc)
        doc.append(NoEscape(
            r"""
            \noindent
            \paragraph{Constans moment around global axis $x$ of intensity
            $m_x$ over a rectangular area}
            """))
        doc.append(expr_to_ltx(r'q_{mn}', sy.latex(qmn_mx), post = ','))
        #doc.append(NoEscape(r"\noindent"))
        doc.append(NoEscape(
            r"""
            where $w$ and $h$ denote the width and height, $x_c$ and $y_c$ the
            coordinates of the center of the rectangle.
            """))

        # constant moment of intensity my, around global axis Y,
        # over a rectangular area with width w, height h and center (xc, yc)
        doc.append(NoEscape(
            r"""
            \noindent
            \paragraph{Constans moment around global axis $y$ of intensity
            $m_y$ over a rectangular area}
            """))
        doc.append(expr_to_ltx(r'q_{mn}', sy.latex(qmn_my), post = ','))
        #doc.append(NoEscape(r"\noindent"))
        doc.append(NoEscape(
            r"""
            where $w$ and $h$ denote the width and height, $x_c$ and $y_c$ the
            coordinates of the center of the rectangle.
            """))

In [ ]:
doc.generate_pdf('e:\\navier', clean_tex=False, compiler='pdfLaTeX')